In [1]:
import cv2
import pytesseract
import streamlit as st
from PIL import Image
import tempfile
import os

# Optional (Windows):
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

st.set_page_config(page_title="Number Plate Reader", layout="centered")
st.title("🚗 Number Plate Detection & OCR")
st.markdown("This app reads number plates from live webcam feed or uploaded images.")

# Sidebar
mode = st.sidebar.radio("Choose Mode", ["📸 Live Webcam", "🖼 Upload Image"])

plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_russian_plate_number.xml')

def detect_plate(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    plates = plate_cascade.detectMultiScale(gray, 1.1, 4)
    detected_texts = []

    for (x, y, w, h) in plates:
        roi = image[y:y+h, x:x+w]
        text = pytesseract.image_to_string(roi, config='--psm 8').strip()
        detected_texts.append(text)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 255, 0), 2)
    
    return image, detected_texts

# 📸 Webcam mode
if mode == "📸 Live Webcam":
    st.info("Press Start to activate your webcam.")

    run = st.checkbox("Start Webcam")
    FRAME_WINDOW = st.image([])

    cap = None
    if run:
        cap = cv2.VideoCapture(0)
        st.warning("Press 'Stop Webcam' to release the camera.")
    else:
        st.stop()

    stop_button = st.button("Stop Webcam")

    while run and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        frame = cv2.flip(frame, 1)
        processed_frame, texts = detect_plate(frame)
        frame_rgb = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
        FRAME_WINDOW.image(frame_rgb)

        if stop_button:
            cap.release()
            break

# 🖼 Upload Image Mode
elif mode == "🖼 Upload Image":
    uploaded_file = st.file_uploader("Upload a car image", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

        processed_image, texts = detect_plate(image_np)
        st.image(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB), caption="Detected Plate")

        if texts:
            st.success(f"Detected Plate Text(s): {', '.join(texts)}")
        else:
            st.error("No number plate detected.")


2025-05-08 19:35:41.751 
  command:

    streamlit run C:\Users\vyvid\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-08 19:35:41.755 Session state does not function when running a script without `streamlit run`
